In [6]:
# load package
import numpy as np
import pandas as pd
from collections import Counter
import lightgbm as lgb
#from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation  import train_test_split
from sklearn import metrics
from sklearn import cross_validation
import pickle
import itertools

# set path
import os
default_path = "/Users/francislin/jrml/What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
with open('all_data.pickle','rb') as f:
    df= pickle.load(f)
# 區分train test
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns
del(df)

Index(['cuisine', 'id', 'used_pepper', 'used_salt', 'used_oil', 'used_garlic',
       'used_ground', 'used_fresh', 'used_sauce', 'used_sugar',
       ...
       'used_snapper', 'used_graham', 'used_tortellini', 'used_bones',
       'used_methi', 'used_cachaca', 'used_hamburger', 'used_candied',
       'used_grapeseed', 'used_superfine'],
      dtype='object', length=702)

In [42]:
dataTrain.dtypes

cuisine             object
id                   int64
used_pepper          int64
used_salt            int64
used_oil             int64
used_garlic          int64
used_ground          int64
used_fresh           int64
used_sauce           int64
used_sugar           int64
used_onions          int64
used_cheese          int64
used_chicken         int64
used_olive           int64
used_black           int64
used_water           int64
used_red             int64
used_flour           int64
used_butter          int64
used_tomatoes        int64
used_green           int64
used_powder          int64
used_chopped         int64
used_cloves          int64
used_juice           int64
used_white           int64
used_onion           int64
used_eggs            int64
used_rice            int64
used_cream           int64
                     ...  
used_prunes          int64
used_slaw            int64
used_blue            int64
used_semi            int64
used_table           int64
used_substitute      int64
u

In [3]:
#dataTrain = dataTrain
#去掉y
dataTest = dataTest.drop('cuisine',axis=1)
#去掉key
X_feature = dataTrain.drop(['id','cuisine'],axis=1)
y_t =dataTrain['cuisine']

In [4]:
#把上面dataTrain.columns的結果貼到dataTrain，再去掉'Survived'
from sklearn.model_selection import train_test_split
#拿掉feature_importances低的因子
X_train, X_test, y_train, y_test = train_test_split(X_feature, y_t, test_size=0.3, random_state=13)

In [41]:
from scipy.stats import randint as sp_randint
# Random Search
print('Start training...')
estimator = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0) 

param_dist = {"max_depth": [3, 7],
              "learning_rate":random.uniform(0.01, 0.15),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "criterion": ["gini", "entropy"]}

n_iter_search = 2
LGBM_Rand = RandomizedSearchCV(estimator, param_distributions=param_dist,
                                   n_iter=n_iter_search)
LGBM_Rand.fit(X_train, y_train)
print('Best parameters found by grid search are:', LGBM_Rand.best_params_)

Start training...


TypeError: object of type 'float' has no len()

In [34]:
# Final Model
evals_result = {} 
print('Start predicting...')
LGBM_Rand_final = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0,
                              criterion = LGBM_Rand.best_params_['criterion'],
                                min_samples_split = LGBM_Rand.best_params_['min_samples_split'], 
                             min_samples_leaf = LGBM_Rand.best_params_['min_samples_leaf'],
                             max_depth = LGBM_Rand.best_params_['max_depth']
                                    )
LGBM_Rand_final_fit = LGBM_Rand_final.fit(X_train, y_train)
print("Predicting is over")

Start predicting...
Predicting is over


In [36]:
y_pred= LGBM_Rand_final_fit.predict(X_test)

In [37]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.65178571, 0.44970414, 0.7424594 , 0.80470588, 0.6367713 ,
       0.60471204, 0.73841555, 0.87669801, 0.51592357, 0.83873487,
       0.72473868, 0.76092545, 0.77657267, 0.90678825, 0.76102088,
       0.44736842, 0.72944585, 0.55309735, 0.78403275, 0.61971831])

0.6961809539587648

0.7796027822006202